<a target="_blank" href="https://colab.research.google.com/github/rajshah4/LLM-Evaluation/blob/main/ragas_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Welcome to the ragas quickstart. We're going to get you up and running with ragas as qickly as you can so that you can go back to improving your Retrieval Augmented Generation pipelines while this library makes sure your changes are improving your entire pipeline.

to kick things of lets start with the data

In [1]:
%pip -q install ragas pandas datasets

Note: you may need to restart the kernel to use updated packages.


Ragas also uses OpenAI for running some metrics so make sure you have your openai key ready and available in your environment

In [1]:
import os
import getpass

open_ai_key = getpass.getpass('Enter your OPENAI API Key')
os.environ['OPENAI_API_KEY'] = open_ai_key

Enter your OPENAI API Key ········


## The Data

Ragas performs a `ground_truth` free evaluation of your RAG pipelines. This is because for most people building a gold labeled dataset which represents in the distribution they get in production is a very expensive process.

Hence to work with ragas all you need are the following data
- question: `list[str]` - These are the questions you RAG pipeline will be evaluated on.
- answer: `list[str]` - The answer generated from the RAG pipeline and give to the user.
- contexts: `list[list[str]]` - The contexts which where passed into the LLM to answer the question.

Ideally your list of questions should reflect the questions your users give, including those that you have been problamatic in the past.

Here we're using an example dataset from on of the baselines we created for the [Financial Opinion Mining and Question Answering (fiqa) Dataset](https://sites.google.com/view/fiqa/) we created. If you want to want to know more about the baseline, feel free to check the `experiements/baseline` section

In [2]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

/Users/leiyu/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating baseline split: 100%|███████████████████████████████| 30/30 [00:00<00:00, 1036.91 examples/s]


DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [18]:
fiqa_eval['baseline']

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [14]:
fiqa_eval['baseline'][0].keys()

dict_keys(['question', 'ground_truths', 'answer', 'contexts'])

In [28]:
fiqa_eval['baseline'][0]

{'question': 'How to deposit a cheque issued to an associate in my business into my business account?',
 'ground_truths': ["Have the check reissued to the proper payee.Just have the associate sign the back and then deposit it.  It's called a third party cheque and is perfectly legal.  I wouldn't be surprised if it has a longer hold period and, as always, you don't get the money if the cheque doesn't clear. Now, you may have problems if it's a large amount or you're not very well known at the bank.  In that case you can have the associate go to the bank and endorse it in front of the teller with some ID.  You don't even technically have to be there.  Anybody can deposit money to your account if they have the account number. He could also just deposit it in his account and write a cheque to the business."],
 'answer': '\nThe best way to deposit a cheque issued to an associate in your business into your business account is to open a business account with the bank. You will need a state-is

## Metrics

Ragas measures your pipeline's performance against two dimensions

1. Faithfulness: measures the factual consistency of the generated answer against the given context.
2. Relevancy: measures how relevant retrieved contexts and the generated answer are to the question.

Through repeated experiments, we have found that the quality of a RAG pipeline is highly dependent on these two dimensions. The final `ragas_score` is the harmonic mean of these two factors.

now lets import these metrics and understand more about what they denote

In [25]:
import ragas

print(ragas.__version__)

0.1.14


In [26]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

here you can see that we are using 4 metrics, but what do the represent?

1. answer_relevancy - a measure of how relevant the answer is to the question

2. faithfulness - the factual consistency of the answer to the context base on the question.

3. context_recall: measures the ability of the retriever to retrieve all the necessary information needed to answer the question.

4. context_precision - a measure of how relevant the retrieved context is to the question. Conveys quality of the retrieval pipeline.

**Note:** *by default these metrics are using OpenAI's API to compute the score. If you using this metric make sure you set the environment key `OPENAI_API_KEY` with your API key. You can also try other LLMs for evaluation, check the [llm guide](./guides/llms.ipynb) to learn more*

If you're interested in learning more, feel free to check the [docs](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

## Evaluation

Running the evalutation is as simple as calling evaluate on the `Dataset` with the metrics of your choice.

In [29]:
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(5)),
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ],
)

result

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|███████████████████████████████████████████████████████| 20/20 [00:15<00:00,  1.26it/s]


{'context_precision': 1.0000, 'faithfulness': 0.8950, 'answer_relevancy': 0.9400, 'context_recall': 0.7778}

and there you have the it, all the scores you need. `ragas_score` gives you a single metric that you can use while the other onces measure the different parts of your pipeline.

now if we want to dig into the results and figure out examples where your pipeline performed worse or really good you can easily convert it into a pandas array and use your standard analytics tools too!

In [30]:
df = result.to_pandas()
df.head()

,question,ground_truths,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,Have the check reissued to the proper payee.Ju...,1.0,0.600,0.982531,0.888889
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,1.0,0.875,0.995228,1.000000
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,1.0,1.000,0.948697,1.000000
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...",1.0,1.000,0.878832,1.000000
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...,You should probably consult an attorney. Howev...,1.0,1.000,0.894728,0.000000


And thats it!

You can check out the [ragas in action] notebook to get a feel of what is like to use it while trying to improve your pipelines.

if you have any suggestion/feedbacks/things your not happy about, please do share it in the [issue section](https://github.com/explodinggradients/ragas/issues). We love hearing from you 😁